# Lab 7-1: Tips

Author: Seungjae Lee (이승재)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Model and Data

In [3]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [4]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [5]:
model = LinearRegressionModel()

In [6]:
def train(model, optimizer, x_train, y_train):
    nb_epochs=1000
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 20번마다 로그 출력
        if epoch % 100 == 99:
            print('Epoch {:4d}/{} Cost: {:.6f}'.format(
                epoch+1, nb_epochs, cost.item()
            ))

## Learning Rate

Gradient Descent 에서의 $\alpha$ 값

`optimizer = optim.SGD(model.parameters(), lr=0.1)` 에서 `lr=0.1` 이다

learning rate이 너무 크면 diverge 하면서 cost 가 점점 늘어난다 (overshooting).

In [7]:
model = LinearRegressionModel()

In [8]:
optimizer = optim.SGD(model.parameters(), lr=1e10)

In [9]:
train(model, optimizer, x_train, y_train)

Epoch  100/1000 Cost: nan
Epoch  200/1000 Cost: nan
Epoch  300/1000 Cost: nan
Epoch  400/1000 Cost: nan
Epoch  500/1000 Cost: nan
Epoch  600/1000 Cost: nan
Epoch  700/1000 Cost: nan
Epoch  800/1000 Cost: nan
Epoch  900/1000 Cost: nan
Epoch 1000/1000 Cost: nan


learning rate이 너무 작으면 cost가 거의 줄어들지 않는다.

In [10]:
model = LinearRegressionModel()

In [11]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [12]:
train(model, optimizer, x_train, y_train)

Epoch  100/1000 Cost: 24799.750000
Epoch  200/1000 Cost: 24777.947266
Epoch  300/1000 Cost: 24756.156250
Epoch  400/1000 Cost: 24734.392578
Epoch  500/1000 Cost: 24712.638672
Epoch  600/1000 Cost: 24690.894531
Epoch  700/1000 Cost: 24669.171875
Epoch  800/1000 Cost: 24647.464844
Epoch  900/1000 Cost: 24625.773438
Epoch 1000/1000 Cost: 24604.101562


적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자.

In [13]:
model = LinearRegressionModel()

In [14]:
optimizer = optim.SGD(model.parameters(), lr=1e-5)

In [15]:
train(model, optimizer, x_train, y_train)

Epoch  100/1000 Cost: 0.267423
Epoch  200/1000 Cost: 0.266821
Epoch  300/1000 Cost: 0.266243
Epoch  400/1000 Cost: 0.265660
Epoch  500/1000 Cost: 0.265089
Epoch  600/1000 Cost: 0.264528
Epoch  700/1000 Cost: 0.263967
Epoch  800/1000 Cost: 0.263421
Epoch  900/1000 Cost: 0.262884
Epoch 1000/1000 Cost: 0.262342


## Data Preprocessing (데이터 전처리)

데이터를 zero-center하고 normalize하자.

In [16]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

$$ x'_j = \frac{x_j - \mu_j}{\sigma_j} $$

여기서 $\sigma$ 는 standard deviation, $\mu$ 는 평균값 이다.

In [17]:
mu = x_train.mean(dim=0)

In [18]:
sigma = x_train.std(dim=0)

In [19]:
norm_x_train = (x_train - mu) / sigma

In [20]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


Normalize와 zero center한 X로 학습해서 성능을 보자

In [21]:
model = LinearRegressionModel()

In [22]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [23]:
train(model, optimizer, norm_x_train, y_train)

Epoch  100/1000 Cost: 0.122644
Epoch  200/1000 Cost: 0.106512
Epoch  300/1000 Cost: 0.106047
Epoch  400/1000 Cost: 0.105922
Epoch  500/1000 Cost: 0.105811
Epoch  600/1000 Cost: 0.105708
Epoch  700/1000 Cost: 0.105609
Epoch  800/1000 Cost: 0.105521
Epoch  900/1000 Cost: 0.105439
Epoch 1000/1000 Cost: 0.105359


## Overfitting

너무 학습 데이터에 한해 잘 학습해 테스트 데이터에 좋은 성능을 내지 못할 수도 있다.

이것을 방지하는 방법은 크게 세 가지인데:

1. 더 많은 학습 데이터
2. 더 적은 양의 feature
3. **Regularization** (일반화)

Regularization: Let's not have too big numbers in the weights

In [24]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs=1000
    for epoch in range(nb_epochs):

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm 계산
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        cost += l2_reg

        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # 20번마다 로그 출력
        if epoch % 100 == 99:
            print('Epoch {:4d}/{} Cost: {:.6f}'.format(
                epoch+1, nb_epochs, cost.item()
            ))

In [25]:
model = LinearRegressionModel()

In [26]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [27]:
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch  100/1000 Cost: 184.486572
Epoch  200/1000 Cost: 184.483932
Epoch  300/1000 Cost: 184.483917
Epoch  400/1000 Cost: 184.483917
Epoch  500/1000 Cost: 184.483917
Epoch  600/1000 Cost: 184.483917
Epoch  700/1000 Cost: 184.483932
Epoch  800/1000 Cost: 184.483917
Epoch  900/1000 Cost: 184.483917
Epoch 1000/1000 Cost: 184.483917
